In [69]:
import numpy as np
import pandas as pd
    from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.layers.core import Dense, Dropout, Activation, Lambda
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [70]:
import keras
print(keras.__version__)

2.1.6


In [71]:
# # Preprocessing:
# # Done Part (I guess :P)
# 1. There were hindi text so "transliteration"
# 2. "|" -> "."
# 3. multiple "..." -> " "
# 4. multiple "    " -> " "
# 5. Removing URLs



# # More preprocccing to do
# 1. Removing stop words (we need to find stope words here first as this is code-mix not normal language)
# 2. removing punctuations from our text
# 3. Removing numbers like phone numbers and all
# 4. changing words like "hai." -> "hai" and "hai..." -> "hai"
# 5. Lowercasing (I guess thats done)
# 6. removing emoticons, but using their info (like angry and disgust emogis and all)


In [72]:
# data.tsv is the file having text and tags after steps of pre processing

df = pd.read_csv('data.tsv',sep = '\t', header=0)

In [73]:
# our main file has 12000 data pair, so some lost here you can see
df.shape

(11634, 2)

In [74]:
# Agressive/abusive words in code mixed
aggressive_words = \
	['Badir','Badirchand','Bakland','Bhandava','Bhoot-nee ka','Chinaal','Chup', \
	'Chutia','Chutiya','chootia','Chutan','Ghasti','gashti','gasti','ghassad','Haraami', \
	'HaraamZaada','Hijda','Hijra','Jaanvar','Kutta','Kuttiya','Khota','Khotey ki aulad', \
	'Kutte ki jat','Najayaz','Najayaz paidaish','Saala kutta','Saali kutti','Soover','Tatti', \
	'Maderchod','Bhosadike','Bhenchod','Betichod','Bhadhava','Chodu','Chutiya','Gaand','Gaandu', \
	'Gadha','Bakland','Lauda','Lund','Hijra','Kuttiya','Paad','Randi','Kamina','Chut','Cuntmama', \
	'Chhed','Bhenkelaude','Bhenketakke','Bhonsri-Waalaa','Bhadwe','Bhains','Buddha Khoosat','Bur', \
	'Chunni','Chinaal','Chudai','Chudan','Gaandfat','Jhat','tatte','Lavde','Lund','Moot', \
	'Najayaz','Rundi','Sadi','sada', 'suar', 'andhe', 'ghulam', 'chaplus', 'madharchod', 'jhaat', 'jhaath', 'ched', 'cheed', \
    'besharam', 'harami', 'chut', 'lund', 'lund', 'keede', 'haramkhor', 'thukwaynge', 'chamcha']

In [75]:
# df.head
train_X = [str(x).lower() for x in df['text']]

In [76]:
train_Y = [str(x) for x in df['tag']]

In [77]:
print(train_X, train_Y)

['mahmood ghaznavi aor abdali ko bhol gaya ha tu \\u0001f602 1 hazar saal tjhy ghulam bana kr rkha tha bhol gaya tu', 'bhai 60sal pehle desh me kya tha pehle pta kro desh waise ka waise kaise hai', 'chutiya friday ko isliye releae krte kyoki week end ke 3 din milte h to log free hokr film dekh sake !! hd h yrr chutiyapa ki', 'jai modiraj', 'upa walo ne bahot kuch kr diya tha desh k liye', 'pan ko aadhar se link ki zarurat kuy hai? supreme court ne ye bola hi nahi', 'kaakar pathar jod ke masjid lai banaay. ta chadhi mullah baangh de , kya baharaa hua khudaay. yah main nahin kabir ne kaha tha.', 'us raat tere papa k jageh mera sperm gya tha use of technology aur tere papa ko pta v na chala \\u0001f602\\u0001f602\\u0001f602 chutiya', 'gater ke keede', 'waise bandhu jet lag se bachne ke liye raat ko 10 baje ke baad so jao', 'get your love back +91-(9815215009) and #all_problems_solution_with_in_72_hours one call change life samasya chaahe kisi bhi ho jad se khatm. badi se badi samasya kaa 

In [78]:
# X = [x.lower for x in train_X]

In [79]:
# type(X)

In [80]:
tokenizer = Tokenizer(num_words=4000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

In [81]:
# giving all the 4k top words a index to be used in the feature set later as ML me numbers use hote hai not text. 
#  So kind of writing a sentence as a sequence of index of words that are there in the sequence.
tokenizer.fit_on_texts(train_X)

In [82]:
print(tokenizer.word_index)

{'ly': 2558, 'siwraj': 19849, 'mujrim': 7815, 'grahaklesh': 29555, 'peke': 15671, 'roz': 1519, 'u0001f622': 998, 'sambit': 5886, 'haalt': 18139, 'massej': 3610, 'u0001f602ni': 30274, 'bandhan': 6828, 'daar': 4587, 'shafiq': 22109, 'bhado': 23830, 'durbhagyapurna': 15169, 'aadhaar': 2764, 'aazam': 7423, 'jarahy': 15313, 'pujan': 12907, 'wlo': 8735, 'khy': 17593, 'metirial': 28727, 'babaao': 30088, 'pulsar': 8686, 'diwali': 5574, 'thar': 2152, 'blasts': 15217, 'khule': 3472, 'district': 21306, 'kill': 13079, 'taaja': 17918, 'lya': 8097, 'super': 2587, 'honichaahiye': 18571, 'haroop': 26245, 'krayega': 29744, 'kdi': 2880, 'guruji': 4621, 'tii': 2707, 'thukta': 16768, 'andr': 4058, 'aapka': 1827, 'innova': 30301, 'urja': 18703, 'hotehi': 29473, 'yogdan': 20684, 'mnon': 23668, "'virodh'": 16478, 'laluji': 6177, 'rahege': 2077, 'cht': 19430, 'aakhen': 19048, 'barana': 21185, 'naga': 9248, 'schoolhe': 30953, 'baithak': 4581, 'ahi': 6706, 'srisriravishankar': 19680, 'trike': 31745, 'alhumdulli

In [83]:
len(train_X)

11634

In [84]:
X = tokenizer.texts_to_sequences(train_X)

In [85]:
sum_tot = 0
for seq in X:
    sum_tot += len(seq)

print(float(sum_tot/11634.0))

new_x = []
new_y = []

for i in range(0, len(X)):
    if len(X[i]) < 400:
        new_x.append(X[i])
        new_y.append(train_Y[i])


24.29628674574523


In [86]:
# The above step because when we pad our data it converts all data into length of the largest data.
# Normally we have one or two data sequence of length 5k but avg. sentence have length of 24-25.
# so we reject some sentences from our data to make our feature set good and small.

In [87]:
# new_X has the data sequence with length less than 400
new_x

[[2159, 3, 58, 67, 80, 42, 122, 153, 177, 82, 41, 58, 80],
 [50, 125, 26, 13, 19, 41, 125, 471, 273, 26, 921, 14, 921, 183, 1],
 [413,
  3,
  319,
  360,
  726,
  2,
  142,
  76,
  1370,
  20,
  7,
  43,
  343,
  344,
  170,
  436,
  20,
  2160,
  6],
 [73],
 [266, 25, 1006, 33, 82, 56, 41, 26, 23, 31],
 [1810, 3, 5, 1688, 6, 1334, 1, 1566, 669, 25, 18, 397, 17, 15],
 [886, 3571, 2, 981, 861, 2060, 68, 19, 83, 147, 10, 40, 25, 102, 41],
 [164,
  581,
  161,
  1689,
  23,
  398,
  254,
  41,
  186,
  306,
  27,
  161,
  1689,
  3,
  471,
  154,
  38,
  594,
  42,
  42,
  42,
  413],
 [2],
 [921, 443, 5, 2, 31, 581, 3, 179, 1300, 2, 128, 670, 331],
 [3572,
  1690,
  1371,
  3286,
  644,
  3899,
  263,
  1420,
  2491,
  716,
  100,
  1978,
  1691,
  1455,
  1372,
  1051,
  903,
  683,
  62,
  12,
  8,
  1567,
  5,
  378,
  311,
  5,
  311,
  903,
  22,
  1070,
  74,
  424,
  683,
  1007,
  3,
  2492,
  78,
  2061,
  47,
  2250,
  5,
  1335,
  887,
  824,
  10,
  1373,
  644,
  3899],
 [5,

In [88]:
print(len(new_x))
print(len(new_y))

11617
11617


In [89]:
# So we lost only 17 data points that's ok
#  as using those 17 wold have made our feature vector as 11634 X 5000 whish has ery high volume of useless 0s.

In [90]:
# padding makes all the sequnces with same length as equal to the largest sequence, by adding by default 0 to the places where no element is there.
#  like if sabse bada sequence 5 size ka hai to sabki size 5 ka bana dega with adding 0 to their rest places jo nahi the abhi tk.
X = pad_sequences(new_x)

In [91]:
# shape here is good to go and make our neural net run quick and good
X.shape

(11617, 390)

In [92]:
# this changes a variable with n values into n-dimentional vector.
#  like say y has value of 0/1, so this does is y=[_,_], index 0 will be 1 if y = 0, or index 1 will be 1 when y = 1.
Y = pd.get_dummies(new_y)

In [93]:
# print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [94]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
# see that the shape of Y's is x3, this is because we have 3 possible values of output.

(9293, 390) (9293, 3)
(2324, 390) (2324, 3)


In [95]:
# Deep Neural Network "MLP": multi layer Perceptron
model = Sequential()

model.add(Dense(256, input_dim=X_train.shape[1]))


# 0.41 : Acc
# model.add(Dense(1000, activation='relu'))
# model.add(Dense(1000, activation='relu'))
# model.add(Dense(1000, activation='relu'))
# model.add(Dense(1000, activation='relu'))
# model.add(Dense(1000, activation='relu'))
# model.add(Dense(1000, activation='relu'))
# model.add(Dense(3, activation='softmax'))



# 0.42 accuracy.
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print("Training...")
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2)

print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

# Evaluating the model
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 32)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Training...
Train on 7434 samples, validate on 1859 samples
Epoch 1/5
7434/7434 [==============================] - 1s 89us/step - loss: 9.4764 - acc: 0.4009 - val_loss: 9.3220 - val_acc: 0.4126
Epoch 2/5
7434/7434 [==============================] - 0s 55us/step - loss: 9.4483 - acc: 0.4034 - val_loss: 9.2378 - val_acc: 0.4190
Epoch 3/5
7434/7434 [==============================] - 0s 56us/step - loss: 9.6794 - acc: 0.3901 - val_loss: 9.3109 - val_acc: 0.4051
Epoch 4/5
7434/7434 [==============================] - 0s 55us/step - loss: 9.3905 - acc: 0.4005 - val_loss: 8.8270 - val_acc: 0.4341
Epoch 5/5
7434/7434 [==============================] - 0s 55us/step - loss: 9.2658 - acc: 0.4096 - val_loss: 9.0198 - val_acc: 0.4244
Generating test predictions...
Score: 9.09
Validation Accuracy: 0.42


In [96]:
preds

array([0, 0, 0, ..., 0, 2, 2])

In [97]:
Y_test

,CAG,NAG,OAG
5062,0,0,1
8704,0,0,1
7204,0,1,0
4265,0,0,1
6605,1,0,0
4762,0,1,0
4420,0,0,1
6338,1,0,0
4436,1,0,0
7469,1,0,0


In [98]:
len(Y_test)

2324

In [99]:
tmpMat = np.zeros((len(Y_test), 3), dtype=int)

In [100]:
for i in range(0,len(predict)):
    if(preds[i]==0):
        tmpMat[i][0]=1
    elif(preds[i]==1):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1

In [103]:
print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))

0.41509367173258466
0.42168674698795183
0.39977415657515186


In [102]:
test_y = Y_test.as_matrix()

# OF CAG
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))


# Of class NAG
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

# Of class OAG
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

0.543821569759283
0.5154905335628227
0.5167317031567354
0.7426698607823101
0.7977624784853701
0.753397958182655
0.5243293869006749
0.5301204819277109
0.526815795927823


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [29]:
# CNN: ConvNeuralNets

nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

In [30]:
from keras.layers.convolutional import Convolution1D
from keras import backend as K

print('Build model...')
model = Sequential()
model.add(Embedding(4000, 128, dropout=0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))

def max_1d(X_train):
    return K.max(X_train, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


Build model...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=250, padding="valid", activation="relu", strides=1, kernel_size=3)`
  del sys.path[0]


In [31]:
print('Train...')
model.fit(X_train, Y_train, batch_size=100, epochs=2,
          validation_split=0.2)
score, acc = model.evaluate(X_test, Y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 7434 samples, validate on 1859 samples
Epoch 1/2
7434/7434 [==============================] - 21s 3ms/step - loss: 0.6055 - acc: 0.6638 - val_loss: 0.5585 - val_acc: 0.6961
Epoch 2/2
2324/2324 [==============================] - 1s 524us/step
Test score: 0.5103624810860613
Test accuracy: 0.7327883076339499


In [32]:
#  CNN Model: test accuracy = 0.73 (1 epochs)
# with 5 epochs Acc: = 0.71

In [ ]:
predict = model.predict(X_test)

In [56]:
test_y = Y_test.as_matrix()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [53]:
predict

array([[0.7907131 , 0.02177265, 0.336867  ],
       [0.20747592, 0.03484862, 0.6200705 ],
       [0.5457721 , 0.12307214, 0.33041114],
       ...,
       [0.6871956 , 0.15370908, 0.19434652],
       [0.07025007, 0.9438393 , 0.01323722],
       [0.11608925, 0.89694   , 0.04299115]], dtype=float32)

In [54]:
tmpMat = np.zeros((len(Y_test), 3), dtype=int)

for i in range(0,len(predict)):
    if(predict[i][0] > predict[i][1] and predict[i][0] > predict[i][2]):
        tmpMat[i][0]=1
    elif(predict[i][1] > predict[i][0] and predict[i][1] > predict[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1
        


In [57]:
print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))

0.6009072409647955
0.5933734939759037
0.5923854997029253


In [58]:
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))

0.6533243234790227
0.6376936316695353
0.6404195052063372


In [59]:
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

0.8323237491941501
0.8412220309810671
0.8357228933368961


In [60]:
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

0.7031445499286377
0.7078313253012049
0.7029302916471879


In [38]:
# LSTM Model for prediction
embed_dim = 128
lstm_out = 196
batch_size = 100

model = Sequential()
model.add(Embedding(4000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 390, 128)          512000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 591       
Total params: 767,391
Trainable params: 767,391
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 2, verbose = 2)

Epoch 1/2
 - 60s - loss: 0.9839 - acc: 0.4653
Epoch 2/2
 - 58s - loss: 0.8106 - acc: 0.6169


In [40]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.88
Validation Accuracy: 0.58


In [41]:
# LSTM Test accuracy = 0.58

In [61]:
p = model.predict(X_test)

In [62]:
p

array([[0.5327422 , 0.08101979, 0.38623804],
       [0.52358896, 0.04492649, 0.43148455],
       [0.617419  , 0.08727141, 0.29530966],
       ...,
       [0.48407108, 0.0776945 , 0.43823448],
       [0.01144527, 0.9812888 , 0.0072659 ],
       [0.01149986, 0.97784334, 0.01065671]], dtype=float32)

In [63]:
tmpMat = np.zeros((len(Y_test), 3), dtype=int)

for i in range(0,len(predict)):
    if(p[i][0] > p[i][1] and p[i][0] > p[i][2]):
        tmpMat[i][0]=1
    elif(p[i][1] > p[i][0] and p[i][1] > p[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1

In [64]:
test_y = Y_test.as_matrix()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [65]:
# For LSTM model
print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))

0.5821903045277752
0.5796041308089501
0.5800862874122301


In [66]:
# Of Class CAG
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))

0.6348859735489983
0.6299483648881239
0.6317142155154932


In [67]:
# Of class NAG
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

0.8341932141294858
0.8343373493975904
0.8342650928112055


In [68]:
# Of class OAG
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

0.691356738496598
0.6949225473321858
0.6923626395420346
